## 에지 검출 & 미분

### 에지

- 배경과 객체 or 객체와 객체의 경계. **픽셀 값이 급격하게 변경**되는 부분. 

   - ex) 어두운 영역에서 갑자기 밝아짐.
   
![image](https://user-images.githubusercontent.com/44194558/138580301-01427184-8079-480d-acb9-65cd41a38eef.png)



- 에지 검출은 객체의 윤곽을 알 수 있는 유용한 방법. 디양한 컴퓨터 비전 태스크에서 객체 판별을 위한 전처리 작업으로 활용됨.

<br/>

에지를 검출하기 위해서는 **픽셀 값의 변화율을 측정(미분)** 하여 변화율이 큰 픽셀을 선택해야 함. 픽셀은 연속 공간 안에 있지 않기 때문에, 영상으로부터 미분을 근사하기 위해서는 다음과 같은 사항을 고려해야 함.

1. 영상은 2차원 평면에서 정의된 함수 -> 가로, 세로 방향 각각 미분 (편미분)


2. 영상은 정수 단위 좌표에 픽셀이 나열되어 있는 이산함수

아래와 같이 2차원 영상 I(x, y)에 대해 x, y축 각각의 방향에 대한 편미분을 중앙 차분 방법으로 근사화. 

![image](https://user-images.githubusercontent.com/44194558/138580456-a6ee3e04-c490-457a-9668-21bc6ef59863.png)

중앙 차분을 이용한 미분 근사는 **마스크 연산**을 이용하여 구현이 가능.

![image](https://user-images.githubusercontent.com/44194558/138580519-fa2cd1c6-833b-4ed8-ac6b-9b6cec8a8795.png)





#### 다양한 미분 마스크

![image](https://user-images.githubusercontent.com/44194558/138580542-6fd092a3-5639-44d4-baa2-6d8cea0cb5b8.png)

<br/>

**Sobel** 

* 현재 행에 대한 중앙 차분 연산을 2회 수행 (더 큰 가중치 부여) & 이전 행, 다음 행에 대해서도 중앙 차분 연산을 1회씩 수행

* 현재 행, 이웃 행에서의 픽셀 값 변화가 유사하다는 점을 이용하여 잡음을 줄임.

**Sobel Filter를 이용한 미분 함수 코드**

cv2.Sobel(src, ddepth, dx, dy, dst=None, ksize=None, scale=None, delta=None, borderType=None) -> dst

* `ddepth` : 출력 영상 데이터 타입. -1이면 입력 영상과 같은 타입 이용

* `dx` : x 방향 미분 차수

* `ksize` : 마스크 커널 크기. default=3

* `scale` : 연산 결과에 추가적으로 곱할 값. default=1

* `delta` : 연산 결과에 추가적으로 더할 값. default=0

* `borderType` : 가장자리 픽셀 확장 방식. default=cv2.BORDER_DEFAULT

**Scharr Filter를 이용한 미분 함수 코드**

cv2.Scharr(src, ddepth, dx, dy, dst=None, ksize=None, scale=None, delta=None, borderType=None) -> dst

* `ddepth` : 출력 영상 데이터 타입. -1이면 입력 영상과 같은 타입 이용

* `dx` : x 방향 미분 차수

* `ksize` : 마스크 커널 크기. default=3

* `scale` : 연산 결과에 추가적으로 곱할 값. default=1

* `delta` : 연산 결과에 추가적으로 더할 값. default=0

* `borderType` : 가장자리 픽셀 확장 방식. default=cv2.BORDER_DEFAULT

#### 영상 미분

In [3]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch06/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

dx = cv2.Sobel(src, -1, 1, 0, delta=128)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()

cv2.destroyAllWindows()

## 그래디언트

함수 f(x, y)를 각각의 방향으로 편미분 하여 벡터 형태로 표현

![image](https://user-images.githubusercontent.com/44194558/138581072-9c280bde-b7d8-479d-af9b-1e5a686ec7ad.png)

**그래디언트 크기** : 픽셀 값의 차이 정도, 변화량. 변화율의 세기에 대한 척도.

![image](https://user-images.githubusercontent.com/44194558/138581088-dbebf9f9-006b-477f-a7e2-4581d8c708b8.png)

**그래디언트 방향** : 픽셀 값이 가장 급격하게 증가하는 방향. 변화 정도가 가장 큰 방향을 나타냄.

![image](https://user-images.githubusercontent.com/44194558/138581100-8013c308-3e6b-425d-a464-be8fdd185e47.png)

<br/>

![image](https://user-images.githubusercontent.com/44194558/138581144-9d44b483-5a79-403c-82f3-f1635e6df66a.png)

<br/>

**영상에서 에지를 검출하는 기본적인 방법은 그래디언트 크기가 특정 값보다 큰 위치를 찾는 것.**

**예시**

![image](https://user-images.githubusercontent.com/44194558/138581181-6c45bcff-264b-4804-93f7-26255b7b8f5d.png)

<br/>

* 빨간색 화살표 : 그래디언트 벡터. 화살표 방향은 그래디언트 벡터의 방향을 나타냄.

벡터의 방향은 해당 위치에서 밝기가 가장 밝아지는 방향을 가리킴.

그래디언트 벡터의 크기는 픽셀 값(밝기) 차이가 클 수록 크게 나타나기 때문에 a, b보다 c에서 화살표 길이가 더 길게 나타남
(점 c에서 볼 수 있는 각각의 편미분 행렬을 이용하여 크기를 계산할 수 있음)

* 노란색 화살표 : 에지의 방향. (그래디언트 벡터에 수직)

**2D 벡터의 크기 계산 함수**

cv2.magnitude(x, y, magnitude=None) -> magnitude

* `x` : x 좌표 행렬. 실수형

* `magnitude` : 크기 행렬. x와 같은 크기, 타입.

![image](https://user-images.githubusercontent.com/44194558/138581325-5f39f173-3ed9-446b-af66-c693c2973ff8.png)

**2D 벡터의 방향 계산 함수**

cv2.phase(x, y, angle=None, angleInDegrees=None) -> angle

* `angle` : 방향 행렬. 실수형.

* `angleInDegrees` : True : 각도 단위, False : radian 단위



#### 영상의 미분을 이용한 에지 검출

In [ ]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch06/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

# 편미분한 결과를 각각 dx, dy 행렬에 저장
# dx, dy는 src와 크기가 같고, 실수형
dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)  
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)

# 두 미분 행렬을 이용하여 그래디언트 벡터의 크기를 계산
mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0, 255).astype(np.uint8)

# 그래디언트의 크기가 특정 ths보다 큰 픽셀을 에지로 검출
# ths값이 너무 작으면 잡음의 영향도 에지로 검출될 수 있으므로 주의
dst = np.zeros(src.shape[:2], np.uint8)
dst[mag > 120] = 255
#_, dst = cv2.threshold(mag, 120, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

## 캐니 에지 검출

소벨 마스크 기반 에지 검출은 구현이 간단하고, 동작이 빠르지만 **그래디언트 크기 만을 기준으로 에지 픽셀을 검출**하기 때문에 ths값에 민감하고, 에지 픽셀이 두껍게 표현된다는 (단순히 그래디언트 크기가 특정 값보다 큰 픽셀을 선택하여, 에지 근방의 여러 픽셀이 한꺼번에 에지로 선택될 수 있음) 단점이 있음.


캐니는 에지 검출을 **최적화 문제**의 관점으로 접근. 좋은 에지 검출기의 조건은

1. 정확한 검출 : 에지를 검출하지 못하거나, False Positive의 확률을 최소화

2. 정확한 위치 : 실제 에지의 중심을 찾아야 함

3. 단일 에지 : 하나의 에지는 하나의 점

<br/>

**그래디언트 크기와 방향을 모두 고려**하여 좀 더 정확한 에지의 위치를 찾을 수 있음. 캐니 에지 검출의 단계는

**1. 가우시안 필터링** 
 - 영상의 잡음 제거. 영상이 부드러워지면서 에지의 세기가 감소할 수 있기 때문에 적절한 표준 편차를 선택해야 함. 잡음이 심하지 않다면 생략.

**2. 그래디언트 계산**
- 가로, 세로 방향으로 각각 소벨 마스크 필터링을 수행한 후, 그래디언트 크기와 방향을 모두 계산.
- 연산 속도를 위해 L1 norm을 사용하는 경우도 있음. (default=L1)

![image](https://user-images.githubusercontent.com/44194558/138581617-adbefa25-dbcf-4056-9704-3db3b1d60bc1.png)

**3. 비최대 억제**

- 그래디언트 크기가 국지적 최대(local max)인 픽셀만을 에지 픽셀로 설정. 나머지 픽셀은 에지 픽셀에서 제외
- 에지 근방의 여러 픽셀이 한꺼번에 에지로 선택되는 것을 막아 에지가 두껍게 표현되는 현상을 방지.
- 그래디언트 방향에 위치한 2개의 픽셀을 조사하여 국지적 최대를 검사.

**4. 히스테리시스 에지 트래킹**

- 두 개의 임계값 사용. 소벨 에지 검출처럼 하나의 임계값을 사용(이분법)하는 경우 조명이나 임계값의 변경에 에지 검출 결과가 민감하기 때문.
- 그래디언트 크기가 T_High보다 크면 해당 픽셀은 최종적으로 에지로 판단. T_Low보다 작으면 에지 픽셀이 아니라고 판단.
- T_High, T_Low 사이에 있는 약한 에지는, 강한 에지와 연결되어 있는 픽셀만 최종 에지로 선정

![image](https://user-images.githubusercontent.com/44194558/138581651-bc10efc6-97c0-4fbb-8ddb-c6f5e1e71538.png)

**캐니 에지 검출 함수**

cv2.Canny(image, ths1, ths2, edges=None, apertureSize=None, L2gradient=None) -> edges

* `ths1, ths2` : T_High, T_Low

* `edges` :에지 영상

* `apertureSize` : 소벨 연산을 위한 커널 크기. default=3

* `L2gradient` : True-L2 norm 사용. default=False (L1 norm)

In [12]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch06/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

dst = cv2.Canny(src, 50, 150) #T_High=150 / T_Low=50

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

## 직선 검출, 원 검출


영상에서 추출한 에지 정보를 이용하여 영상에서 직선, 원을 검출하는 방법

### 허프 변환 : 직선 검출

직선 검출 : 자율 주행 자동차에서 차선을 검출 or 수평이 맞지 않는 영상에서 수평선이나 수직선 성분을 찾아내어 자동 영상 회전을 위한 정보로 사용

에지를 검출하고, 에지 픽셀들이 일직선상에 배열되어 있는지를 확인해야 함. 허프 변환은 **2차원 영상 좌표에서 직선의 방정식을 따라 파라미터 공간으로 변화하여 직선을 찾는** 알고리즘.


![image](https://user-images.githubusercontent.com/44194558/138581931-e41a8a27-c0f0-410f-b1e9-e191b05f5a1c.png)


2차원 xy 평면에서 직선의 방정식은 파라미터인 기울기 a, 절편 b에 의해 형태가 결정됨. 반대로 ab 평면에서 변환된 직선의 방정식은 기울기가 -x, 절편이 y인 형태로 표현.

**xy 공간에서의 직선은 ab 공간에서 한 점으로 표현되고, xy 공간에서의 한 점은 ab 공간에서 직선으로 표현됨.**

허프 변환을 이용하여 직선의 방정식을 찾으려면 **xy 공간에서 에지로 판별된 모든 점을 이용하여 ab 파라미터 공간에 직선을 표현하고, 직선이 많이 교차되는 좌표를 모두 찾아야 함.** 직선이 많이 교차하는 점을 찾기 위해 축적 배열을 사용하여 2차원 배열에서 직선이 지나가는 위치의 배열 원소값들을 1씩 증가시킴.

![image](https://user-images.githubusercontent.com/44194558/138582196-2df8d525-aae4-4605-9a43-1bde277cc706.png)

<br/>

y=ax+b 형식은 모든 형태의 직선을 표현하기 어렵기 때문에 (ex) 수직선) 보통 **극좌표계** 형식의 직선의 방정식을 사용.

![image](https://user-images.githubusercontent.com/44194558/138582273-c8a8d0a9-5506-4265-a064-be9f921b80b3.png)

극좌표계의 파라미터(theta, p) 공간 변환

![image](https://user-images.githubusercontent.com/44194558/138582295-18d4c6e3-30b8-4c29-8962-51621c56940d.png)

1. xy 좌표계의 3점을 선택


2. p, theta 파라미터 공간에서 3개 점들의 곡선을 표현


3. p, theta 공간에서 세 곡선이 하나의 점에서 모두 교차하는 것을 확인. (교차점이 파란색의 직선의 방정식을 표현하는 파라미터)

**허프 변환에 의한 직선 검출**

cv2.HoughLines(image, rho, theta, ths, lines=None, srn=None, stn=None, min_theta=None, max_theta=None) -> lines

* `image` : 입력 에지 영상
* `rho` : 축적 배열에서 rho 값의 간격
* `theta` : 축적 배열에서 theta 값의 간격
* `ths` :직선으로 판단하는 임계값
* `lines` : 직선의 방정식 파라미터 정보를 담고 있는 numpy.ndarray. shape=(N, 1, 2), dtype=numpy.float32.
* `srn, stn` : 멀티 스케일 허프 변환에서 rho 해상도, theta 해상도를 나누는 값. default=0 (일반 허프 변환)
* `min_theta, max_theta` : 검출한 선분의 최소, 최대 theta값


**확률적 허프 변환에 의한 선분 검출**

직선의 방정식 파라미터 p, theta가 아닌 직선의 시작점과 끝점 좌표를 반환.

cv2.HoughLinesP(image, rho, theta, ths, lines=None, minLineLength=None, maxLineGap=None) -> lines

* `image` : 입력 에지 영상
* `rho` : 축적 배열에서 rho 값의 간격
* `theta` : 축적 배열에서 theta 값의 간격
* `ths` :직선으로 판단하는 임계값
* `lines` : 선분의 시작과 끝 좌표 (x1, y1, x2, y2) 정보를 담고 있는 numpy.ndarray. shape=(N, 1, 4), dtype=numpy.int32.
* `minLineLength` : 검출할 선분의 최소 길이
* `maxLineGap` : 직선으로 간주할 최대 에지 점 간격. 일직선상의 직선이 잡음 등의 영향으로 끊어져 있을 때, 두 직선을 하나의 직선으로 간주하고자 할때 사용

In [ ]:
import sys
import numpy as np
import cv2

src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch06/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

edges = cv2.Canny(src, 50, 150)

lines = cv2.HoughLinesP(edges, 1, np.pi / 180., 160,
                        minLineLength=50, maxLineGap=5)

dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])  # 시작점 좌표
        pt2 = (lines[i][0][2], lines[i][0][3])  # 끝점 좌표
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

### 허프 변환 : 원 검출

원의 방정식은 3개의 파라미터를 가짐. 3차원 공간에서 축적 배열을 정의하고 사용하기엔 메모리와 연산 시간을 많이 필요로 하기 때문에 **그래디언트 방법**을 활용하여 원을 검출함.


**Hough gradient method**

1. **입력 영상과 동일한** 2차원 평면 공간에서 **축적 영상**을 생성 (파라미터 공간 x, 입력과 동일한 xy 좌표 공간의 2차원 배열)

2. 입력 영상의 모든 에지 픽셀에서 그래디언트 계산

3. 에지 방향을 따라 직선을 그리면서 축적 배열의 값 누적, 원의 중심 좌표 찾기.

4. 검출된 원의 중심으로부터 원에 적합한 반지름을 계산


![image](https://user-images.githubusercontent.com/44194558/138582834-b1dcc644-4589-47fe-90f2-d5b9788c245b.png)

* 원주 상의 모든 점에 대해 **그래디언트 방향의 직선**을 그리고, 직선상의 축적 배열 값을 증가 시키면 원의 중심 위치에서 축적 배열 값이 크게 나타남.

* 원의 중심을 찾고 나면, 다양한 반지름의 원에 대해 원주상에 충분히 많은 에지 픽셀이 존재하는 지 확인하여 적절한 반지름을 선택


**단점** : 여러 개의 동심원을 검출하지 못함. 가장 작은 원 하나만 검출.

**허프 변환 원 검출 함수**

cv2.HoughCircles(image, method, dp, minDist, circles=None, param1=None, param2=None, minRadius=None, maxRadius=None) -> circles

* `image` : 입력 영상. **에지 영상이 아닌 일반 영상**.

* `method` : OpenCV 4.2 이하에서는 cv2.HOUGH_GRADIENT만 지정 가능

* `dp` : 입력 영상과 축적 배열의 크기 비율. 1이면 동일 크기, 2이면 축적 배열의 가로, 세로가 입력 영상의 절반.

* `minDist` : 검출된 원 중심점들의 최소 거리

* `circles` : (cx, cy, r) 정보를 담은 numpy.ndarray. shape=(N, 1, 3), dtype=numpy.float32

* `param1` : 캐니 에지 검출기의 T_High. T_Low는 param1의 절반으로 설정.

* `param2` : 축적 배열에서 원 검출을 위한 임계값

* `minRadius, maxRadius` : 검출할 원의 최소, 최대 반지름

In [ ]:
import sys
import numpy as np
import cv2


# 입력 이미지 불러오기
src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch06/dial.jpg')

if src is None:
    print('Image open failed!')
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)  # 잡음 제거용


def on_trackbar(pos):
    rmin = cv2.getTrackbarPos('minRadius', 'img')
    rmax = cv2.getTrackbarPos('maxRadius', 'img')
    th = cv2.getTrackbarPos('threshold', 'img')
    
    # 원 검출. 두 원의 중심점 거리가 50픽셀보다 작으면 검출 x.
    # 축적 배열 원소 값이 30보다 크면 원의 중심점으로 선택.
    # circles는 검출된 원의 중심 좌표, 반지름 정보 저장
    circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,  
                               param1=120, param2=th, minRadius=rmin, maxRadius=rmax)

    dst = src.copy()
    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = circles[0][i]
            cv2.circle(dst, (cx, cy), int(radius), (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('img', dst)


# 트랙바 생성
cv2.imshow('img', src)
cv2.createTrackbar('minRadius', 'img', 0, 100, on_trackbar)
cv2.createTrackbar('maxRadius', 'img', 0, 150, on_trackbar)
cv2.createTrackbar('threshold', 'img', 0, 100, on_trackbar)
cv2.setTrackbarPos('minRadius', 'img', 10)
cv2.setTrackbarPos('maxRadius', 'img', 80)
cv2.setTrackbarPos('threshold', 'img', 40)
cv2.waitKey()

cv2.destroyAllWindows()

## 동전 카운터


영상의 동전을 검출하여 금액을 자동으로 계산.

1. 동전 검출 -> 허프 원 검출

2. 동전 구분 -> 색상 정보 이용

In [ ]:
import sys
import numpy as np
import cv2


# 입력 이미지 불러오기
src = cv2.imread('C:/Users/ky_moon/Desktop/vision/ch06/coins1.jpg')

if src is None:
    print('Image open failed!')
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1)

# 허프 변환 원 검출
# circles.shape=(1, 11, 3). 1 개의 영상에 대해 11개의 원에 대한 정보를 검출
circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,
                           param1=150, param2=40, minRadius=20, maxRadius=80)

# 원 검출 결과 및 동전 금액 출력
sum_of_money = 0
dst = src.copy()
if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)

        # 사각형 모양의 동전 영역 부분 영상 추출
        x1 = int(cx - radius)
        y1 = int(cy - radius)
        x2 = int(cx + radius)
        y2 = int(cy + radius)
        radius = int(radius)
        
        crop = dst[y1:y2, x1:x2, :]
        ch, cw = crop.shape[:2]

        # 동전 영역에 대한 ROI 마스크 영상 생성 (동전 부분만 하얗게, 나머지 배경은 검은색)
        mask = np.zeros((ch, cw), np.uint8)
        cv2.circle(mask, (cw//2, ch//2), radius, 255, -1)

        # 동전 영역 Hue 색 성분을 +40 시프트하고, Hue 평균을 계산
        hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        hue, _, _ = cv2.split(hsv)
        hue_shift = (hue + 40) % 180
        mean_of_hue = cv2.mean(hue_shift, mask)[0]

        # Hue 평균이 90보다 작으면 10원, 90보다 크면 100원으로 간주
        won = 100
        if mean_of_hue < 90:
            won = 10

        sum_of_money += won

        cv2.putText(crop, str(won), (20, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, (255, 0, 0), 2, cv2.LINE_AA)

cv2.putText(dst, str(sum_of_money) + ' won', (40, 80),
            cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()